In [ ]:
1
# Импортирование библиотек
import pandas as pd
import os, warnings, joblib
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Смотрим исходные данные
train = pd.read_csv("df/train.csv", low_memory=False)
valid = pd.read_csv("df/valid.csv", low_memory=False)

arr = [train, valid]
names_df = ['train', 'valid']

# Анализ данных
def analyze_data(df):
    print(df.describe())
    sns.countplot(x='target', data=df)
    plt.title('Distribution of Target Variable')
    plt.show()

for i, elem in enumerate(arr):
    print(f'Анализ данных для {names_df[i]}:')
    analyze_data(elem)

# Остальной код...

# Модель RandomForestClassifier
X = train_updt.drop(columns=['client_id', 'target', 'report_date'])
y = train_updt['target']

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category']).columns

X[categorical_features] = X[categorical_features].astype(str)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=100, class_weight='balanced'))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
model.fit(X_train, y_train)

test_preds = model.predict(X_test)
test_preds_proba = model.predict_proba(X_test)[:, 1]
test_score = roc_auc_score(y_test, test_preds_proba)

print('Test')
print(f"Accuracy: {accuracy_score(y_test, test_preds):.4f}")
print(f"ROC-AUC: {test_score:.4f}")
print(classification_report(y_test, test_preds))

#########################################################################################
perenos_()

X_valid = valid_updt.drop(columns=['client_id', 'target', 'report_date'], errors='ignore')
y_valid = valid_updt['target']
X_valid[categorical_features] = X_valid[categorical_features].astype(str)

valid_preds = model.predict(X_valid)
valid_preds_proba = model.predict_proba(X_valid)[:, 1]
valid_score = roc_auc_score(y_valid, valid_preds_proba)

print('Valid')
print(f"Accuracy: {accuracy_score(y_valid, valid_preds):.4f}")
print(f"ROC-AUC: {valid_score:.4f}")
print(classification_report(y_valid, valid_preds))

# Модель XGBoost
# Т.к. XGBoost не может обрабатывать столбцы типа object (строковые) без предварительной обработки. 
# Сделаем предварительные обработки

# Использование fit_transform для обучающего набора = обучаете 
X_train_transformed = preprocessor.fit_transform(X_train)
# transform для тестового и валидационного наборов = применяете
X_test_transformed = p

reprocessor.transform(X_test)
X_valid_transformed = preprocessor.transform(X_valid)

# Создание и обучение модели XGBoost
model_xgb = XGBClassifier(scale_pos_weight=len(y_train) / sum(y_train == 1),
                      random_state=42,
                      eval_metric='logloss',
                      max_depth=30,
                      n_estimators=500,
                      learning_rate=0.1)

model_xgb.fit(X_train_transformed, y_train)

# Кросс-валидация
cv_scores_xgb = cross_val_score(model_xgb, X_train_transformed, y_train, cv=kf, scoring='roc_auc')
print(f'Кросс-валидация ROC-AUC XGBoost: {cv_scores_xgb.mean():.4f} ± {cv_scores_xgb.std():.4f}')

test_preds_xgb = model_xgb.predict(X_test_transformed)
test_preds_proba_xgb = model_xgb.predict_proba(X_test_transformed)[:, 1]
test_score_xgb = roc_auc_score(y_test, test_preds_proba_xgb)

print('Test XGBoost')
print(f"Accuracy: {accuracy_score(y_test, test_preds_xgb):.4f}")
print(f"ROC-AUC: {test_score_xgb:.4f}")
print(classification_report(y_test, test_preds_xgb))

#########################################################################################
perenos_()

valid_preds_xgb = model_xgb.predict(X_valid_transformed)
valid_preds_proba_xgb = model_xgb.predict_proba(X_valid_transformed)[:, 1]
valid_score_xgb = roc_auc_score(y_valid, valid_preds_proba_xgb)

print('Valid XGBoost')
print(f"Accuracy: {accuracy_score(y_valid, valid_preds_xgb):.4f}")
print(f"ROC-AUC: {valid_score_xgb:.4f}")
print(classification_report(y_valid, valid_preds_xgb))

# Модель LightGBM
# Игнорирование FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning)

# Установка переменной окружения
os.environ["LOKY_MAX_CPU_COUNT"] = "4"

lgb_model = LGBMClassifier(boosting_type='gbdt',
    objective='binary',
    class_weight='balanced',
    learning_rate=0.01, n_estimators=500, num_leaves=20, min_child_samples=30, random_state=42)

lgb_model.fit(X_train_transformed, y_train)

# Кросс-валидация
cv_scores_lgb = cross_val_score(lgb_model, X_train_transformed, y_train, cv=kf, scoring='roc_auc')
print(f'Кросс-валидация ROC-AUC LightGBM: {cv_scores_lgb.mean():.4f} ± {cv_scores_lgb.std():.4f}')

test_preds_lgb = lgb_model.predict(X_test_transformed)
test_preds_proba_lgb = lgb_model.predict_proba(X_test_transformed)[:, 1]
test_score_lgb = roc_auc_score(y_test, test_preds_proba_lgb)

print('\nTest LightGBM')
print(f"Accuracy: {accuracy_score(y_test, test_preds_lgb):.4f}")
print(f"ROC-AUC: {test_score_lgb:.4f}")
print(classification_report(y_test, test_preds_lgb))

#########################################################################################
perenos_()

valid_preds_lgb = lgb_model.predict(X_valid_transformed)
valid_preds_proba_lgb = lgb_model.predict_proba(X_valid_transformed)[:, 1]
valid_score_lgb = roc_auc_score(y_valid, valid_preds_proba_lgb)

print('Valid LightGBM')
print(f"Accuracy: {accuracy_score(y_valid, valid_preds_lgb):.4f}")
print(f"ROC-AUC: {valid_score_lgb:.4f}")
print(classification_report(y_valid, valid_preds_lgb))
```

### Добавленные элементы:
1. **Анализ данных**: Функция `analyze_data` для отображения описательной статистики и распределения целевой переменной.
2. **Метрики**: Использование `classification_report` для более детальной оценки моделей.
3. **Кросс-валидация**: Реализована кросс-валидация для всех моделей с использованием `cross_val_score`.
4. **Интерпретация результатов**: Добавление вывода отчета о классификации для оценки метрик.
5. **Сохранение модели**: Модели сохраняются с помощью `joblib`.

Если нужно внести дополнительные изменения или улучшения, дай знать!


2
> PulseGPT - AI Assistant:
Давайте более подробно рассмотрим, что делает обработка IQR (межквартильный размах) в вашем коде, 
а также предложим возможные улучшения.

### Что делает код

Функция `iqr_remove` выполняет следующие действия:

1. **Получение числовых столбцов**:
   - Если аргумент `columns` не задан, функция выбирает все числовые столбцы из DataFrame.

2. **Копирование DataFrame**:
   - Создается копия исходного DataFrame `df_clean`, чтобы сохранить оригинальные данные нетронутыми.

3. **Расчет квартилей и IQR**:
   - Для каждого столбца вычисляются первый (Q1) и третий (Q3) квартиль, а также IQR (межквартильный размах) как `IQR = Q3 - Q1`.

4. **Определение границ**:
   - Устанавливаются нижняя и верхняя границы для определения выбросов:
     - `lower_bound = Q1 - threshold * IQR`
     - `upper_bound = Q3 + threshold * IQR`
   - Значение `threshold` по умолчанию равно 3, что означает, что все значения ниже `Q1 - 3 * IQR` и выше `Q3 + 3 * IQR` будут считаться выбросами.

5. **Фильтрация данных**:
   - Из DataFrame `df_clean` удаляются все строки, которые имеют значения за пределами установленных границ.

6. **Возврат очищенного DataFrame**:
   - Функция возвращает очищенный DataFrame без выбросов.

### Пример использования

Код ниже выполняет вышеописанную функцию для каждого DataFrame в массиве `arr` и выводит количество строк до и после удаления аномалий:

```python
for i, elem in enumerate(arr):
    cleaned = iqr_remove(elem)
    print(f"{names_df[i]} до удаления аномалий: {elem.shape[0]} строк, {elem.shape[1]} столбцов")
    print(f"{names_df[i]} после удаления аномалий: {cleaned.shape[0]} строк, {cleaned.shape[1]} столбцов")
    cleaned.to_csv(f'df/_{names_df[i]}_updt.csv', index=False)
    print("Данные успешно сохранены.")
    perenos()
```

### Возможные улучшения

1. **Гибкость порога**:
   - Позвольте пользователю задавать значение `threshold` при вызове функции, чтобы адаптировать его под разные наборы данных.

2. **Логирование выбросов**:
   - Сохраните информацию о выбросах, чтобы можно было видеть, какие значения были удалены. Это может помочь в анализе и интерпретации данных.

3. **Обработка пустых значений**:
   - Добавьте обработку NaN значений перед вычислением квартилей, чтобы избежать ошибок.

4. **Альтернативные методы для определения выбросов**:
   - Рассмотрите возможность использования других методов обнаружения выбросов, таких как Z-оценка, для сравнения результатов.

5. **Визуализация**:
   - Добавьте возможность визуализировать данные до и после удаления выбросов с помощью boxplot, чтобы наглядно показать влияние очистки данных.

6. **Обработка категориальных данных**:
   - Если в вашем наборе данных есть категориальные переменные, рассмотрите возможность их обработки или преобразования в числовые значения, прежде чем применять IQR.

### Пример улучшенного кода

Вот обновленный код с учетом предложенных улучшений:

def iqr_remove(df, columns=None, threshold=3):
    if columns is None:
        columns = df.select_dtypes(include=['number']).columns
    
    df_clean = df.copy()
    anomalies = {}
    
    for col in columns:
        # Проверка на NaN значения
        if df[col].isnull().any():
            df[col] = df[col].fillna(df[col].median())  # Заполнение медианой
        
        Q1 = df[col].quantile(0.25)  
        Q3 = df[col].quantile(0.75)  
        IQR = Q3 - Q1  
        
        lower_bound = Q1 - threshold * IQR  
        upper_bound = Q3 + threshold * IQR
        
        # Сохранение выбросов
        anomalies[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        
        # Фильтрация данных
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    
    # Логирование выбросов
    for col, anomaly_data in anomalies.items():
        if not anomaly_data.empty:
            print(f"Выбросы в столбце {col}:")
            print(anomaly_data)

    return df_clean

for i, elem in enumerate(arr):
    cleaned = iqr_re

move(elem)
    print(f"{names_df[i]} до удаления аномалий: {elem.shape[0]} строк, {elem.shape[1]} столбцов")
    print(f"{names_df[i]} после удаления аномалий: {cleaned.shape[0]} строк, {cleaned.shape[1]} столбцов")
    cleaned.to_csv(f'df/_{names_df[i]}_updt.csv', index=False)
    print("Данные успешно сохранены.")
    perenos()

Этот код добавляет обработку NaN значений и логирование выбросов, что делает обработку данных более прозрачной и информативной.